In [0]:
#This notebook achieves 98% accuracies on the Pneumonia picture classification dataset
#Results were not saved
from google.colab import files
files.upload()
#Upload your kaggle.json api key
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [0]:
import zipfile
with zipfile.ZipFile("/content/chest-xray-pneumonia.zip","r") as zip_ref:
    zip_ref.extractall('/content')
with zipfile.ZipFile("/content/chest_xray.zip","r") as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
from torchvision.models import *

from fastai.vision import *
#from fastai.vision.models import *


In [ ]:
!git clone https://github.com/sdoria/mish #SOTA optimizers and activations functions

In [ ]:
%cd /kaggle/working/mish
from rangerlars import *
from mish import *
from mxresnet import *
from ranger import *

In [ ]:
from fastai.callbacks import *
#Annealing
def flattenAnneal(learn:Learner, lr:float, n_epochs:int, start_pct:float):
  n = len(learn.data.train_dl)
  anneal_start = int(n*n_epochs*start_pct)
  anneal_end = int(n*n_epochs) - anneal_start
  phases = [TrainingPhase(anneal_start).schedule_hp('lr', lr),
           TrainingPhase(anneal_end).schedule_hp('lr', lr, anneal=annealing_cos)]
  sched = GeneralScheduler(learn, phases)
  learn.callbacks.append(sched)
  learn.fit(n_epochs)

In [ ]:
%cd /kaggle/working

In [ ]:
path = Path('../input/chest-xray-pneumonia/chest_xray/chest_xray')
path.ls()

In [ ]:
tfms = get_transforms()

In [ ]:
bs= 64

In [ ]:
np.random.seed(1337)
size=256
data = ImageDataBunch.from_folder(path,
                                  valid= 'val', valid_pct=0.2,
                                  size=size, bs=bs,
                                  ds_tfms=tfms).normalize(imagenet_stats)
#data = ImageList.from_folder(path).split_by_folder(valid='val').label_from_folder().transform(([flip_lr(p=0.5)], []), size=size).databunch(bs=bs).presize(size, scale=(0.35,1)).normalize(imagenet_stats)

In [ ]:
data.show_batch(3, figsize=(6,6))

In [ ]:
data.c, data.classes, len(data.train_ds), len(data.valid_ds)

In [ ]:

# opt_func = partial(Ranger, eps=1e-6)
act_fn = Mish()
learn = cnn_learner(data, models.resnet50, metrics=error_rate, model_dir="/tmp/model/")
# learn = cnn_learner(data, mxresnet34(c_out=2, sa=True), metrics=error_rate, model_dir="/tmp/model/")

In [ ]:
#flattenAnneal(learn, 4e-3, 5, 0.72)
learn.fit_one_cycle(4)

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(3e-5, 3e-4))